In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta, time
from DynamicTimeModel import DynTM
from Qeuryalgorithm import QHalgorithm, path_interpretor
from QHquery import queryAlgorithm

In [2]:
def haversine_affinity(x,y):
    R = 6378137 #in meter

    #convert to raduis
    lat1  = x[0] * np.pi/180
    long1 = x[1] * np.pi/180
    lat2  = y[0] * np.pi/180
    long2 = y[1] * np.pi/180
    
    #calculate haversine distance
    delta_longitude = long1 - long2
    delta_latitude = lat1 - lat2
    a = (np.sin(delta_latitude/2)**2) + np.cos(lat1)*np.cos(lat2)*(np.sin(delta_longitude/2)**2)
    c = 2*np.arctan2(np.sqrt(a),np.sqrt(1-a))
    distance = R*c
    
    return distance

In [3]:
def str_time_to_int(ch):
    ch = ch.split(':')
    H  = int(ch[0])*60*60
    M  = int(ch[1])*60
    S  = int(ch[2])
    Time = (H+M+S)%86400
    return Time

### Import data

In [4]:
timetable = pd.read_csv('../Data preprocessing/data/preprocessed_timetable.csv')
timetable.next_station = timetable.next_station.astype(int)
timetable = timetable.sort_values(['horaires_depart'])

In [5]:
#convert str to time object
timetable.horaires_depart  = timetable.horaires_depart.apply(str_time_to_int)
timetable.horaires_arrivee = timetable.horaires_arrivee.apply(str_time_to_int)

In [6]:
Dmatrix = pd.read_csv('../Data preprocessing/data/Dmatrix.csv')
Dmatrix = Dmatrix.set_index('station_id')

In [7]:
stations_paths = pd.read_csv('../Data preprocessing/data/stations_paths.csv')
stations_coord = pd.read_csv('../Data preprocessing/data/station_coord.csv')
paths = pd.read_csv('../Data preprocessing/data/paths.csv')

In [8]:
stations = pd.concat([timetable.next_station,timetable.prev_station]).reset_index(drop=True).unique()
stations_paths = stations_paths[stations_paths.station_id.isin(stations)]
stations_paths = stations_paths[['station_id', 'english_name', 'french_name', 'arabic_name','latitude','longitude']].drop_duplicates()

In [9]:
stations_coord = stations_coord[stations_coord.station_id.isin(stations)]

## Build DTM Graph Structure

In [10]:
#intiate graph
max_walk_between_stations = 300 #300meters
walk_speed  = 1.4  #1.4 meter per second
DTM = DynTM(max_walk_between_stations, walk_speed)
#generate graph and add unrestricted path
DTM.generateTimetableDynamicModel(timetable)
DTM.add_unresticted_paths(Dmatrix)

In [11]:
DTM.dtm.number_of_nodes()

30209

In [12]:
DTM.dtm.number_of_edges()

84486

## DTM-QH algorithm to solve EA Problem

In [13]:
from time import time as timo

curr_time = datetime.now().time().replace(microsecond=0)
curr_time = str(curr_time.hour)+':'+str(curr_time.minute)+':'+str(curr_time.second)
curr_time = str_time_to_int(curr_time)
curr_time = str_time_to_int('11:10:00')

#source = ('source_node', 36.87007096206944, 10.34382253848957)   #sidi bou said
source = ('source_node', 36.87970216988326, 10.327309234021802)   #marsa ville
#target = ('target_node', 36.8226374125104, 10.30882462463282) #la goulette
#source = ('source_node', 36.83906834458209, 10.316478568193) #le kram
target = ('target_node',36.8066962256805, 10.18126936246436) #siege IT transtu 
#target = ('target_node', 36.806678, 10.181269)   #Tunis marine
#target = ('target_node', 36.829968, 10.150268)  #manar1
#target = ('target_node', 36.724688, 10.205779)  #el mourouj 3
#target = ('target_node',36.85340441371543, 10.196437250933252)  #ariana
#target = ('target_node', 36.81792285359155, 10.080213439106986)  #manouba
#target = ('target_node', 36.75537489487085, 10.22252052118309 ) #Ben Arouss
#target = ('target_node', 36.83848676642428, 10.183395901376352) #Manzeh 1 cité sportive
#target = ('target_node', 36.80936808780992, 10.133522078274913) #Musée bardo
#target = ('target_node', 36.796247434333004, 10.132042159557537) #ezzouhour
#target = ('target_node', 36.79086785239071, 10.092124602977709) #cité el mechtel
#target = ('target_node', 36.841494756981, 10.132237773927486) #Omrane sup
#target = ('target_node', 36.79037011799913, 10.107341203160626) #municipalité hrairia (ezzahrouni)
#target = ('target_node', 36.788855425031464, 10.177730508869596) #beb alioua
#target = ('target_node', 36.839583107126735, 10.117761184660045) #el intilaka


start = timo()
path, transitions, walk = QHalgorithm(DTM.dtm, source, target, curr_time, stations_coord, walk_panalisation=True, transfer_time=150, max_StationDistance = 1000)
stop = timo()
print(stop - start)

0.2144637107849121


In [14]:
walk

[1896]

In [15]:
# target manar1 , 20:10:00
# target manouba, 22:30:00
# siege it transtu 11:10:00

In [16]:
path

[['switch_source',
  ('switch', 89),
  ('node', 11, 89, 'TGM', 41400),
  ('node', 11, 88, 'TGM', 41524),
  ('node', 11, 87, 'TGM', 41606),
  ('node', 11, 86, 'TGM', 41718),
  ('node', 11, 85, 'TGM', 41814),
  ('node', 11, 84, 'TGM', 41900),
  ('node', 11, 83, 'TGM', 42015),
  ('node', 11, 82, 'TGM', 42077),
  ('node', 11, 81, 'TGM', 42150),
  ('node', 11, 80, 'TGM', 42223),
  ('node', 11, 79, 'TGM', 42315),
  ('node', 11, 78, 'TGM', 42390),
  ('node', 11, 77, 'TGM', 42459),
  ('node', 11, 76, 'TGM', 42513),
  ('node', 11, 75, 'TGM', 42601),
  ('node', 11, 74, 'TGM', 42672),
  ('node', 11, 73, 'TGM', 42792),
  ('switch', 72),
  ('switch', 69),
  ('node', 16, 69, '6', 44280),
  ('switch', 70),
  ('switch', 16),
  'switch_target']]

In [17]:
#ken ncost < qcost dima wa77el
#ken ncost => qcost chouf ken tekel wa9t akther ama feha trans w walk a9al wa7el

In [18]:
start = timo()
pathqh, trans, dist = queryAlgorithm(DTM.dtm, source, target, curr_time, stations_coord, walk_panalisation=True, transfer_time=150)
stop = timo()
print(stop - start)
pathqh

0.025930404663085938


['switch_source',
 ('switch', 89),
 ('node', 11, 89, 'TGM', 41400),
 ('node', 11, 88, 'TGM', 41524),
 ('node', 11, 87, 'TGM', 41606),
 ('node', 11, 86, 'TGM', 41718),
 ('node', 11, 85, 'TGM', 41814),
 ('node', 11, 84, 'TGM', 41900),
 ('node', 11, 83, 'TGM', 42015),
 ('node', 11, 82, 'TGM', 42077),
 ('node', 11, 81, 'TGM', 42150),
 ('node', 11, 80, 'TGM', 42223),
 ('node', 11, 79, 'TGM', 42315),
 ('node', 11, 78, 'TGM', 42390),
 ('node', 11, 77, 'TGM', 42459),
 ('node', 11, 76, 'TGM', 42513),
 ('node', 11, 75, 'TGM', 42601),
 ('node', 11, 74, 'TGM', 42672),
 ('node', 11, 73, 'TGM', 42792),
 ('switch', 72),
 ('switch', 15),
 'switch_target']

In [221]:
ncost = 11
qcost = 11
ntrans = 4
qtrans = 5
qwalk_dist = 5
nwalk_dist = 5


if ncost >= qcost :
    if  not (ntrans < qtrans or nwalk_dist < qwalk_dist) :
        print('skip')
    else :
        print('add to quee')
else:
    print('add to quee')

add to quee


### Path verfication

In [19]:
print(path_interpretor(pathqh, stations_paths, paths, language='french'))

aller à pied jusqu'à la station LA MARSA et prendre la ligne (TGM) : Marsa vers Tunis marine à 11H 30m 0s
descendre du véhicule à la station TUNIS MARINE
de la station TUNIS MARINE aller à pied jusqu'à la station TUNIS MARINE
aller à pied jusqu'à votre destination


In [20]:
haversine_affinity(target[1:], [36.812810, 10.089279])

8226.900491814667

In [21]:
haversine_affinity(target[1:], [36.797900, 10.186210])

1073.662262195404

In [22]:
timetable[(timetable.line_id == 16) & (timetable.TripNb == 10)]

,TripNb,line_label,line_id,tag,order,prev_station,next_station,prev_latitude,prev_longitude,next_latitude,next_longitude,horaires_depart,horaires_arrivee,datetime_diff,vehicle_type,distance_diff
9763,10,6,16,0,1,69,70,36.799610,10.193027,36.797900,10.186210,21360,21508,148.0,2,637.0
9968,10,6,16,0,2,70,14,36.797900,10.186210,36.796109,10.180552,21508,21679,171.0,2,542.0
4461,10,6,16,0,3,14,106,36.796109,10.180552,36.785807,10.179813,21679,21878,199.0,2,1149.0
15847,10,6,16,0,4,106,107,36.785807,10.179813,36.782100,10.179840,21878,21990,112.0,2,413.0
16171,10,6,16,0,5,107,108,36.782100,10.179840,36.775040,10.179200,21990,22128,138.0,2,788.0
16495,10,6,16,0,6,108,109,36.775040,10.179200,36.767880,10.183880,22128,22251,123.0,2,900.0
16818,10,6,16,0,7,109,116,36.767880,10.183880,36.760650,10.187016,22251,22400,149.0,2,852.0
17769,10,6,16,0,8,116,117,36.760650,10.187016,36.754668,10.187660,22400,22510,110.0,2,668.0
17977,10,6,16,0,9,117,118,36.754668,10.187660,36.748444,10.188733,22510,22615,105.0,2,699.0
18184,10,6,16,0,10,118,119,36.748444,10.188733,36.745177,10.190492,22615,22681,66.0,2,396.0
